# Balanço Patrimonial do Estado do Espírito Santo 🏢

### Projeto desenvolvido no Curso Bootcamp Análise de Dados 2023, promovido pela ENAP
#### Equipe:
* Ana Paula Giancoli
* Eugênia Giancoli
* Gislaine Messias
* Mariana Hermínia

### Importando as bibliotecas

In [1]:
import numpy as np
import os, csv, pandas as pd
from datetime import date
import requests
import time
import streamlit as st
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import plotly.express as px
import plotly.io as pio
from sklearn import preprocessing

### Função de carregamento da pasta do projeto

In [2]:
def build_path(subfolder = 'data'):
    folderpath = os.path.join(os.getcwd(), 
                              'projeto', subfolder)
    folderpath = os.path.abspath(folderpath)
    if not os.path.exists(folderpath): 
        os.makedirs(folderpath)
    return folderpath

### Função de carregamento da Planilha PCASP - Plano de Contas aplicado ao Setor Público
* Selecionando o arquivo excel de nome: CPU_PCASP_2022.xlsx
* Selecionando a planilha de nome: Federação 2022

###### Observações: 
* É necessário copiar o arquivo de Plano de Contas para dentro da pasta do projeto com o nome CPU_PCASP_2022.xls.
* A linha @st.cahce_data deve ser descomentada somente no arquivo.py para ser executada.

In [3]:
#@st.cache_data
def get_dados_pcasp(
        filename='CPU_PCASP_2022.xlsx'):
    filepath = os.path.join(build_path(), filename)
    return pd.read_excel(filepath, sheet_name= 'Federação 2022')

### Função de carregamento da Matriz de Saldos Contábeis

In [4]:
#@st.cache_data
def get_dados_instancia(filename='instancia_MSC_API.csv'):
    filepath = os.path.join(build_path(), filename)
    return pd.read_csv(filepath, sep=';')

### Função de download da Matriz de Saldos Contábeis do site do SICONFI de 2019 a 2022 e do ente 32, que é o Estado do Espírito Santo
* Mês de referência é o mês 12.
* Ente Federativo é o código 32, que se refere ao Estado do Espírito Santo.
* Tipo de matriz é Matriz de encerramento MSCE.
* Classe de contas: tipos 1 e 2.
* Tipo do valor é o ending_balance.

###### Observação:
* Verificar a documentação disponível em [Swagger-API](https://apidatalake.tesouro.gov.br/docs/siconfi/#/MSC%20-%20Patrimonial/get_msc_patrimonial).

In [5]:
def download_matriz(id_ente, ano_referencia, classe_conta):
    url='https://apidatalake.tesouro.gov.br/ords/siconfi/tt/msc_patrimonial'
    params={'id_ente':f'{id_ente}', 'an_referencia': f'{ano_referencia}','me_referencia':'12','co_tipo_matriz':'MSCE','classe_conta': f'{classe_conta}','id_tv':'ending_balance'}
    response = requests.get(url,params=params).json()
    return response

### Função para salvar a Matriz de Saldos Contábeis no arquivo `instancia_MSC_API.csv` na pasta do projeto

In [6]:
def save_matriz(matriz):
    arquivo = os.path.join(build_path(), "instancia_MSC_API.csv")
    matriz.to_csv(arquivo, index=False, sep=';')

### Função para buscar os dados da Matriz de Saldos Contábeis executando a função de download, em seguida, salva na pasta do projeto com o nome de arquivo `instancia_MSC_API.csv`

In [7]:
def search_datas(id_ente = 32, years = [2019, 2020, 2021, 2022], classes = [1, 2]):    
    responses = []
    for year in years:
        for class_ in classes:
            resp = download_matriz(id_ente, year, class_)
            responses.append(resp['items'])
            time.sleep(5)
    
    df_final = pd.DataFrame(responses[0])

    for i in range(1,len(responses)):
        df = pd.DataFrame(responses[i])
        df_final = pd.concat([df_final, df], ignore_index=True, axis=0)
        
    save_matriz(df_final)
    return df_final

### Carregamento da Matriz de Saldos Contábeis

In [8]:
search_datas()

,tipo_matriz,cod_ibge,classe_conta,conta_contabil,poder_orgao,financeiro_permanente,ano_fonte_recursos,fonte_recursos,exercicio,mes_referencia,divida_consolidada,data_referencia,entrada_msc,valor,natureza_conta,tipo_valor,complemento_fonte
0,MSCE,32,1,111110100,10111,1.0,1.0,10900000,2019,12,None,2019-12-31T03:00:00Z,2,14667.39,D,ending_balance,None
1,MSCE,32,1,111115099,10111,1.0,1.0,10900000,2019,12,None,2019-12-31T03:00:00Z,4,500.00,D,ending_balance,None
2,MSCE,32,1,111110200,10111,1.0,1.0,19790000,2019,12,None,2019-12-31T03:00:00Z,6,7671.79,D,ending_balance,None
3,MSCE,32,1,111110200,10111,1.0,1.0,10010000,2019,12,None,2019-12-31T03:00:00Z,8,69723.95,D,ending_balance,None
4,MSCE,32,1,111110200,10111,1.0,1.0,12110000,2019,12,None,2019-12-31T03:00:00Z,10,2679.20,D,ending_balance,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6579,MSCE,32,2,237130300,10111,NaN,NaN,None,2022,12,None,2022-12-31T03:00:00Z,3838,23251.40,D,ending_balance,None
6580,MSCE,32,2,237130300,10112,NaN,NaN,None,2022,12,None,2022-12-31T03:00:00Z,3840,921829.00,C,ending_balance,None
6581,MSCE,32,2,237210100,10111,NaN,NaN,None,2022,12,None,2022-12-31T03:00:00Z,3844,3330221.17,C,ending_balance,None
6582,MSCE,32,2,237210200,10111,NaN,NaN,None,2022,12,None,2022-12-31T03:00:00Z,3846,10235770.20,C,ending_balance,None


### Carregamento da Planilha PCASP - Plano de Contas aplicado ao Setor Público
* Variável pcasp

In [9]:
pcasp = get_dados_pcasp()

### Removendo colunas da Planilha PCASP - Plano de Contas aplicado ao Setor Público que não serão utilizadas

In [10]:
pcasp = pcasp.drop(columns=['FUNÇÃO','STATUS','NATUREZA DO SALDO','CONTROLE','ÍTEM','SUBÍTEM'])

### Selecionando as classes 1 e 2 da Planilha PCASP - Plano de Contas aplicado ao Setor Público que serão analisadas

In [11]:
pcasp = pcasp.loc[pcasp['CLASSE'].isin([1, 2])]

### Renomeando as colunas da  Planilha PCASP - Plano de Contas aplicado ao Setor Público que estão com o nome extenso

In [12]:
pcasp.loc[pcasp['TÍTULO.1'] == 'OBRIGAÇÕES TRABALHISTAS, PREVIDENCIÁRIAS E ASSISTENCIAIS A PAGAR A LONGO PRAZO','TÍTULO.1'] = 'OBRIGAÇÕES TRAB., PREV. E ASSIST. A PAGAR A LONGO PRAZO'

pcasp.loc[pcasp['TÍTULO.1'] == 'OBRIGAÇÕES TRABALHISTAS, PREVIDENCIÁRIAS E ASSISTENCIAIS A PAGAR A CURTO PRAZO','TÍTULO.1'] = 'OBRIGAÇÕES TRAB., PREV. E ASSIST. A PAGAR A CURTO PRAZO'

### Criando as colunas NÍVEL3, NÍVEL2, NÍVEL1 na variável pcasp, como strings a partir das CLASSES da Planilha PCASP - Plano de Contas aplicado ao Setor Público 

* Coluna pcasp['NÍVEL3']

* Coluna pcasp['NÍVEL2']

* Coluna pcasp['NÍVEL1']

In [13]:
pcasp['NÍVEL3'] = pcasp['CLASSE'].astype(str) + pcasp['GRUPO'].astype(str) + pcasp['SUBGRUPO'].astype(str)

pcasp['NÍVEL2'] = pcasp['CLASSE'].astype(str) + pcasp['GRUPO'].astype(str)

pcasp['NÍVEL1'] = pcasp['CLASSE'].astype(str)

### Criando o Dataframe por NÍVEL
* **pcasp_nivel_3**
    * pcasp_nivel_3, com TODAS as contas de NÍVEL3 do PCASP para criar o Balanço NÍVEL3
    * São as contas do PCASP que estão indicadas nas contas de cores em azul

* **pcasp_nivel_2**
    * pcasp_nivel_2, com TODAS as contas de NÍVEL2 do PCASP para criar o Balanço NÍVEL2
    * São as contas do PCASP que estão indicadas nas contas de cores em verde

* **pcasp_nivel_1**
    * pcasp_nivel_1, com TODAS as contas de NÍVEL1 do PCASP para criar o Balanço NÍVEL1
    * São as contas do PCASP que estão indicadas nas contas de cores em vermelho

In [14]:
pcasp_nivel_3 = pcasp.loc[(pcasp['SUBGRUPO']!= 0) & (pcasp['TÍTULO']== 0)]

pcasp_nivel_2 = pcasp.loc[(pcasp['GRUPO']!= 0) & (pcasp['SUBGRUPO']== 0)]

pcasp_nivel_1 = pcasp.loc[(pcasp['GRUPO']== 0)]

### Criando o Dataframe pcasp_nivel_1_2_3, com TODAS as contas de NÍVEL1, NÍVEL2 e NÍVEL3 do PCASP para criar o Balanço NÍVEL3
* São todas as contasdo PCASP que estão indicadas nas contas de cores em vermelho, verde e azul

In [15]:
pcasp_nivel_1_2_3 = pd.concat([pcasp_nivel_1,pcasp_nivel_2,pcasp_nivel_3],axis=0)

### Carregamento da Matriz de Saldos Contábeis
* Variável instancia

In [17]:
instancia = get_dados_instancia()

### Removendo colunas da Matriz de Saldos Contábeis

In [18]:
instancia = instancia.drop(columns=['tipo_matriz','cod_ibge','poder_orgao','financeiro_permanente','ano_fonte_recursos','fonte_recursos','mes_referencia','divida_consolidada','data_referencia','entrada_msc','tipo_valor','complemento_fonte'])

### Renomeando as colunas da Matriz de Saldos Contábeis

In [19]:
instancia = instancia.rename(columns={'classe_conta':'NÍVEL1', 'conta_contabil':'CONTA', 'exercicio': 'EXERCÍCIO', 'valor':'VALOR', 'natureza_conta':'NATUREZA_VALOR'})

### Alterando o tipo da coluna CONTA para string

In [20]:
instancia['CONTA'] = instancia['CONTA'].astype(str)

### Aplicando filtros para identificar as CLASSES dos GRUPOS 1 e 2
* Multiplica a coluna VALOR por 1 para contas de natureza Devedora (Classe 1)
* Multiplica a coluna VALOR por -1 para contas de natureza Credora (Classe 1)
* Multiplica a coluna VALOR por 1 para contas de natureza Credora (Classe 2)
* Multiplica a coluna VALOR por -1 para contas de natureza Devedora (Classe 2)
* Consequentemente, na soma, os valores de débito serão somados e os valores de crédito serão subtraídos (RETIFICADORAS)

In [21]:
#  Multiplica por 1 - Desnecessária
instancia.loc[instancia['NÍVEL1'].isin([1]) & 
              (instancia['NATUREZA_VALOR']=='D'), 'VALOR'] = instancia['VALOR']*1

#  Multiplica por -1
instancia.loc[instancia['NÍVEL1'].isin([1]) & 
              (instancia['NATUREZA_VALOR']=='C'), 'VALOR'] = instancia['VALOR']*(-1)

#  Multiplica por 1 - Desnecessária
instancia.loc[instancia['NÍVEL1'].isin([2]) & 
              (instancia['NATUREZA_VALOR']=='C'), 'VALOR'] = instancia['VALOR']*1

#  Multiplica por -1
instancia.loc[instancia['NÍVEL1'].isin([2]) & 
              (instancia['NATUREZA_VALOR']=='D'), 'VALOR'] = instancia['VALOR']*(-1)

### Criando Colunas

* Criando a coluna NÍVEL3 na Matriz de Saldos Contábeis a partir da identificação dos três primeiros caracteres das CONTAS para agrupar as CONTAS do NÍVEL3
    * instancia['NÍVEL3']
* Criando a coluna NÍVEL2 na Matriz de Saldos Contábeis a partir da identificação dos dois primeiros caracteres das CONTAS para agrupar as CONTAS do NÍVEL2
    * instancia['NÍVEL2']
* Criando a coluna NÍVEL1 na Matriz de Saldos Contábeis 
    * instancia['NÍVEL1']

In [22]:
instancia['NÍVEL3'] = instancia['CONTA'].str.slice(0, 3)

instancia['NÍVEL2'] = instancia['CONTA'].str.slice(0, 2)

coluna = [col for col in instancia if col != 'NÍVEL1'] + ['NÍVEL1']
instancia = instancia[coluna]

### Agrupando as CONTAS do NÍVEL3
* Armazenando em instancia_grouped

In [23]:
instancia_grouped = instancia[['VALOR']].groupby([instancia['EXERCÍCIO'], instancia['NÍVEL3'], instancia['NATUREZA_VALOR']])

### Criando o Dataframe:

* **NÍVEL3 - instancia_soma_nivel_3**
    * instancia_soma_nivel_3, com TODOS os valores somados por NÍVEL3, NATUREZA e ANO(EXERCÍCIO)
    * Armazenando em instancia_soma_nivel_3
    * Totalizando os valores por nível, somando os débitos e créditos


* **NÍVEL2 - instancia_soma_nivel_2**
    * instancia_soma_nivel_2, com TODOS os valores somados por NÍVEL2, NATUREZA e ANO(EXERCÍCIO)
    * Armazenando em instancia_soma_nivel_2
    * Totalizando os valores por nível, somando os débitos e créditos
    
    
* **NÍVEL1 - instancia_soma_nivel_1**
    * instancia_soma_nivel_1, com TODOS os valores somados por NÍVEL1, NATUREZA e ANO(EXERCÍCIO)
    * Armazenando em instancia_soma_nivel_1
    * Totalizando os valores por nível, somando os débitos e créditos

In [24]:
instancia_soma_nivel_3 = instancia[['VALOR']].groupby([instancia['EXERCÍCIO'], instancia['NÍVEL3']]).sum()

instancia_soma_nivel_2 = instancia[['VALOR']].groupby([instancia['EXERCÍCIO'], instancia['NÍVEL2']]).sum()

instancia_soma_nivel_1 = instancia[['VALOR']].groupby([instancia['EXERCÍCIO'], instancia['NÍVEL1']]).sum()

### Criando a coluna com EXERCÍCIO e NÍVEL3 (que são índices) nos respectivos Dataframe:  
* em instancia_soma_nivel_3, 
* em instancia_soma_nivel_2,
* em instancia_soma_nivel_1. 

In [25]:
instancia_soma_nivel_3 = instancia_soma_nivel_3.reset_index(level=['NÍVEL3','EXERCÍCIO'])

instancia_soma_nivel_2 = instancia_soma_nivel_2.reset_index(level=['NÍVEL2','EXERCÍCIO'])

instancia_soma_nivel_1 = instancia_soma_nivel_1.reset_index(level=['NÍVEL1','EXERCÍCIO'])

### Alterando o tipo da coluna NÍVEL1 para string¶

In [26]:
instancia_soma_nivel_1['NÍVEL1'] = instancia_soma_nivel_1['NÍVEL1'].astype(str)

### Combinando as instancias e pcasp por NÍVEL para criar os Balanços por NÍVEL

* NÍVEL3 (balanco_nivel_3) - Combinando o resultado das operações da INSTANCIA de NÍVEL3 com a coluna correspondente do PCASP de NÍVEL3
* NÍVEL2 (balanco_nivel_2)- Combinando o resultado das operações da INSTANCIA de NÍVEL2 com a coluna correspondente do PCASP de NÍVEL2
* NÍVEL1 (balanco_nivel_1)- Combinando o resultado das operações da INSTANCIA de NÍVEL1 com a coluna correspondente do PCASP de NÍVEL1

In [27]:
balanco_nivel_3 = pd.merge(pcasp_nivel_3, instancia_soma_nivel_3, how='inner')

balanco_nivel_2 = pd.merge(pcasp_nivel_2, instancia_soma_nivel_2, how='inner')

balanco_nivel_1 = pd.merge(pcasp_nivel_1, instancia_soma_nivel_1, how='inner')

### Ordenando os balancos de NÍVEL por ANO(EXERCÍCIO) e CONTA
* Ordenando balanco_nivel_3 por ANO(EXERCÍCIO) e CONTA
* Ordenando balanco_nivel_2 por ANO(EXERCÍCIO) e CONTA
* Ordenando balanco_nivel_1 por ANO(EXERCÍCIO) e CONTA

In [28]:
balanco_nivel_3 = balanco_nivel_3.sort_values(by=['EXERCÍCIO','CONTA'])

balanco_nivel_2 = balanco_nivel_2.sort_values(by=['EXERCÍCIO','CONTA'])

balanco_nivel_1 = balanco_nivel_1.sort_values(by=['EXERCÍCIO','CONTA'])

### Removendo as colunas dos Balanços de NÍVEL
* Removendo as colunas balanco_nivel_3
* Removendo as colunas balanco_nivel_2
* Removendo as colunas balanco_nivel_1

In [29]:
balanco_nivel_3 = balanco_nivel_3.drop(columns=['CLASSE','GRUPO','SUBGRUPO','TÍTULO','SUBTÍTULO'])

balanco_nivel_2 = balanco_nivel_2.drop(columns=['CLASSE','GRUPO','SUBGRUPO','TÍTULO','SUBTÍTULO'])

balanco_nivel_1 = balanco_nivel_1.drop(columns=['CLASSE','GRUPO','SUBGRUPO','TÍTULO','SUBTÍTULO'])

### Criando uma única INSTANCIA com todos os NÍVEIS - balanco_123

#### Criando o Balanço balanco_123 concatenando TODOS os NÍVEIS das INSTANCIAS de NÍVEL1, NÍVEL2 e NÍVEL3

In [30]:
balanco_123 = pd.concat([balanco_nivel_1,balanco_nivel_2,balanco_nivel_3],axis=0)

#### Ordenando balanco_123 por ANO(EXERCÍCIO) e CONTA

In [31]:
balanco_123 = balanco_123.sort_values(by=['EXERCÍCIO','CONTA'])

#### Removendo as colunas de balanco_123 que não são necessárias

In [32]:
balanco_123 = balanco_123.drop(columns=['NÍVEL3','NÍVEL2','NÍVEL1'])

### Criando o Dataframe com o Balanço Final - balanco_final que será utilizado para a criação do gráfico Treemap por meio do balanco_normalizado

In [33]:
lista_exercicios = list(set(balanco_123['EXERCÍCIO']))
balanco = []
for ano in lista_exercicios:
    balanco_ano = balanco_123.loc[balanco_123['EXERCÍCIO'] == ano]
    balanco_ano = balanco_ano.rename(columns={'VALOR': f'{ano}'})
    balanco_ano = balanco_ano.drop(columns=['EXERCÍCIO'])
    balanco.append(balanco_ano)
balanco_final = pd.merge(balanco[0], balanco[1], how='outer', on=['CONTA','TÍTULO.1'])
balanco_final = pd.merge(balanco_final, balanco[2], how='outer', on=['CONTA','TÍTULO.1'])
balanco_final = pd.merge(balanco_final, balanco[3], how='outer', on=['CONTA','TÍTULO.1'])
balanco_final = balanco_final.sort_values(by='CONTA')

### Criando o balanco_final_exercicio que é a Transposta de balanco_final, onde as linhas serão os anos(exercícios) e as colunas serão as contas para ser utilizado na criação do gráfico da Série Histórica

In [34]:
balanco_final_exercicio = balanco_final.T

In [35]:
lista = []
for i in range(len(balanco_final)):
    lista.append( list(balanco_final_exercicio.loc['TÍTULO.1'])[i])

In [36]:
balanco_final_exercicio = balanco_final_exercicio.rename(columns=balanco_final_exercicio.iloc[1])
balanco_final_exercicio = balanco_final_exercicio[2:]
balanco_final_exercicio.index.names = ['EXERCÍCIO']
balanco_final_exercicio = balanco_final_exercicio.reset_index(level=['EXERCÍCIO'])

In [37]:
balanco_final['CONTA'] = balanco_final['CONTA'].astype(str)
balanco_final['CLASSE'] = balanco_final['CONTA'].str.slice(0,1)
balanco_final = balanco_final.rename(columns={2019: '2019', 2020: '2020', 2021: '2021', 2022: '2022'})

In [38]:
balanco_normalizado = balanco_final.copy()

### A listaAnos é gerada a partir das opções selecionadas na sidebar para montar o gráfico de todos os anos - Série Histórica a partir da MSC
* Criada a coluna ANO+ABS para os anos que possuem valores negativos. Assim o gráfico Treemap é exibido corretamente por meio da normalização.

In [39]:
# Foi fixada para efeitos de execução
listaAnos = ['2019', '2020', '2021', '2022' ]
for ano in listaAnos:
    coluna_ano = listaAnos[listaAnos.index(ano)]
    if balanco_final[coluna_ano].min() < 0 :
        listanorm = preprocessing.normalize([balanco_normalizado[coluna_ano].values.tolist()]) 
        coluna_ano = coluna_ano+'ABS'
        listanormal = listanorm-(listanorm.min()*2)
        balanco_normalizado[coluna_ano] = pd.DataFrame(listanormal).T

### Criado o balanco_final_map para organizar a exibição do gráfico treemap com as subcontas classificadas adequadamente

In [40]:
balanco_final_map = balanco_final.copy()

#### Criar colunas em balanco_final para organizar o treemap nas suas subcontas
* Criar coluna que indica se a conta é Ativo ou Passivo e Patrimônio Líquido 
    * balanco_final_map['N1']

In [41]:
balanco_final_map['N1']=' '

#Ativo
balanco_final_map.loc[balanco_final_map['CONTA'].str.slice(0,1)== '1', 'N1'] = 'ATIVO'

#Passivo e PL
balanco_final_map.loc[balanco_final_map['CONTA'].str.slice(0,1) == '2', 'N1'] ='PASSIVO E PATRIMÔNIO LIQUIDO'

### Classificando corretamente cada conta
* Para conta que são as próprias N1, deixe o campo vazio
    * balanco_final.loc[balanco_final['CONTA'].str.slice(1,2) == '0', 'N1'] =' '
* Para conta que são subcontas de N1 e N2, deixe o campo vazio
    * balanco_final.loc[balanco_final['CONTA'].str.slice(2,3) != '0', 'N1'] =' '

* Criar coluna que indica de a conta é Ativo(Ativo Circulante ou Ativo Não Circulante) ou Passivo e PL(Passivo Circulante ou Passivo Não Circulante ou PL)
    * balanco_final_map['N2']
    
* Para conta que são as próprias N1 e N2, deixe o campo vazio
    * balanco_final.loc[balanco_final['CONTA'].str.slice(2,3) == '0', 'N2'] =' '

In [42]:
balanco_final_map['N2']=' '

#Ativo Circulante
balanco_final_map.loc[balanco_final_map['CONTA'].str.slice(0,2) == '11', 'N2'] = 'ATIVO CIRCULANTE'

#Ativo Não Circulante
balanco_final_map.loc[balanco_final_map['CONTA'].str.slice(0,2) == '12', 'N2'] = 'ATIVO NÃO CIRCULANTE'

#Passivo Circulante
balanco_final_map.loc[balanco_final_map['CONTA'].str.slice(0,2) == '21', 'N2'] ='PASSIVO CIRCULANTE'

#Passivo Não Circulante
balanco_final_map.loc[balanco_final_map['CONTA'].str.slice(0,2) == '22', 'N2'] ='PASSIVO NAO-CIRCULANTE'

#PL
balanco_final_map.loc[balanco_final_map['CONTA'].str.slice(0,2) == '23', 'N2'] ='PATRIMÔNIO LIQUIDO'

### balanco_final_map

In [43]:
balanco_final_map = balanco_final_map[ (balanco_final_map['TÍTULO.1'] != 'ATIVO') & (balanco_final_map['TÍTULO.1'] != 'PASSIVO E PATRIMÔNIO LIQUIDO') & (balanco_final_map['TÍTULO.1'] != 'ATIVO CIRCULANTE') & (balanco_final_map['TÍTULO.1'] != 'ATIVO NÃO CIRCULANTE') & (balanco_final_map['TÍTULO.1'] != 'PASSIVO CIRCULANTE') & (balanco_final_map['TÍTULO.1'] != 'PASSIVO NAO-CIRCULANTE') & (balanco_final_map['TÍTULO.1'] != 'PATRIMÔNIO LIQUIDO')]

### Código adicionado no BP_ES_2019-2022.py

In [44]:
# st.markdown('# Balanço Patrimonial do Estado do Espírito Santo 🏢')
# st.markdown("#### Série Histórica 2019-2022 a partir da MSC ")
# st.markdown("---")

# with st.sidebar:
#     #escolha_ente = st.sidebar.selectbox('Escolha o Ente:', ('ES', ''))
#     st.sidebar.markdown('# Balanço Patrimonial do Estado do Espírito Santo 🏢')
#     st.sidebar.markdown("---")
#     st.markdown("#### Série Histórica 2019-2022 a partir da MSC:")
#     st.sidebar.markdown("##### Escolha um ou mais anos: ")
#     ano2019=st.sidebar.checkbox("2019", value=True)
#     ano2020=st.sidebar.checkbox("2020")
#     ano2021=st.sidebar.checkbox("2021")
#     ano2022=st.sidebar.checkbox("2022")
#     listaAnos=[]
#     if ano2019:
#         listaAnos.append("2019")
#     if ano2020:
#         listaAnos.append("2020")
#     if ano2021:
#         listaAnos.append("2021")
#     if ano2022:
#         listaAnos.append("2022")

#     st.sidebar.markdown("---")
#     st.markdown(f'#### Série Histórica 2019-2022 selecionada pela Conta ')
#     escolha_descr = st.sidebar.selectbox('Escolha uma das contas: ', balanco_final['TÍTULO.1'])

    
# # Gráfico Geral com Série histórica por contas
# chart_data = pd.DataFrame(balanco_final, columns=listaAnos)
# st.bar_chart(chart_data)
# st.markdown("---")



# st.bar_chart(balanco_final, y=listaAnos, x='CONTA', color=listaAnos[0])
# st.markdown("---")  


# st.markdown("#### Série Histórica 2019-2022 selecionada pela Conta ")
# col1, col2 = st.columns([3, 8])
# with col1:
#     pass

# with col2:
#     st.markdown(f"###### {escolha_descr} ")
#     balanco = pd.DataFrame(balanco_final_exercicio, columns=['EXERCÍCIO', escolha_descr])
   

# st.bar_chart(balanco_final_exercicio, y=escolha_descr,x='EXERCÍCIO',color=escolha_descr)
# st.markdown("---")  

In [45]:

# for ano in listaAnos:
#     # Gráfico Treemap ATIVO                             
#     ativo = px.treemap(balanco_final_map.loc[balanco_final_map['CLASSE'] == '1'], 
#                      path = ['N1','N2', 'TÍTULO.1'], 
#                      values = listaAnos[listaAnos.index(ano)], 
#                      color_continuous_scale='RdBu',
#                      color = listaAnos[listaAnos.index(ano)],             
#                      color_continuous_midpoint=0)

#     ativo.update_layout(margin = dict(t=50, l=25, r=25, b=25))   
#     coluna_ano = listaAnos[listaAnos.index(ano)]
    
#     if balanco_final_map[coluna_ano].min() < 0 :        
#         listanorm = preprocessing.normalize([balanco_normalizado[coluna_ano].values.tolist()]) 
#         coluna_ano = coluna_ano+'ABS'
#         listanormal = listanorm-(listanorm.min()*2)
#         balanco_normalizado[coluna_ano] = pd.DataFrame(listanormal).T
    
#     # Gráfico Treemap PASSIVO
#     passivo = px.treemap(balanco_normalizado.loc[balanco_normalizado['CLASSE'] == '2'], 
#                      path = ['N1','N2', 'TÍTULO.1'], 
#                      values = coluna_ano, 
#                      color_continuous_scale='RdBu',
#                      color = listaAnos[listaAnos.index(ano)],             
#                      color_continuous_midpoint=np.average(balanco_normalizado[coluna_ano]))

#     passivo.update_layout(margin = dict(t=50, l=25, r=25, b=25))
#     st.markdown("---")
    
#     # Atualizar legenda dos TreeMaps
#     ativo.update_layout(title_text='Mapa do Ativo de ' + listaAnos[listaAnos.index(ano)], title_font=dict(size=24), title_x = 0.025)
#     passivo.update_layout(title_text='Mapa do Passivo e Patrimônio Líquido de ' + listaAnos[listaAnos.index(ano)], title_font=dict(size=24), title_x = 0.025)
    
#     # Layout com duas colunas
#     col1, col2 = st.columns(2)

#     # Colocar TreeMaps nas colunas
#     with col1:
#         st.plotly_chart(ativo, use_container_width=True)

#     with col2:
#         st.plotly_chart(passivo, use_container_width=True)